In [1]:
import openai
from openai.embeddings_utils import get_embedding,cosine_similarity
import pandas as pd
import re
import tiktoken
import os 
from dotenv  import load_dotenv
import json

In [2]:
openai.api_type = "azure"
openai.api_base = ""
openai.api_version = "2023-03-15-preview"
openai.api_key = ""

In [3]:
# def get_embedding_test(query):
#     # Call OpenAI API to get the embeddings.
#     response = openai.Embedding.create(
#         input=query,
#         # Your deployment name
#         engine="asmitaembedding" ## deployment name in azure open ai playground --> deployments 
#     )
#     embeddings = response['data'][0]['embedding']
#     # print(embeddings)
#     # print(response)
#     return response

In [3]:
# get some data 
df = pd.read_csv("data/data-embedding.csv")
df.head()

,title,description,label_int,label
0,World Briefings,BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime M...,1,World
1,Nvidia Puts a Firewall on a Motherboard (PC Wo...,PC World - Upcoming chip set will include buil...,4,Sci/Tech
2,"Olympic joy in Greek, Chinese press",Newspapers in Greece reflect a mixture of exhi...,2,Sports
3,U2 Can iPod with Pictures,"SAN JOSE, Calif. -- Apple Computer (Quote, Cha...",4,Sci/Tech
4,The Dream Factory,"Any product, any shape, any size -- manufactur...",4,Sci/Tech


In [4]:
def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

df['description'] = df["description"].apply(lambda x : normalize_text(x))

In [5]:
print("=====================================")
embeddings_1 = get_embedding(df["description"][51],engine = "asmitaembedding")
print(df["description"][51])
check_txt=df["description"][51]
print(f"{type(check_txt)}")
print('Array Size (Short Text) - Ada: ', len(embeddings_1))
print(embeddings_1)

AP - From LSU at No. 1 to Ohio State at No. 10, The AP women's basketball poll had no changes Monday.
<class 'str'>
Array Size (Short Text) - Ada:  1536
[-0.007328147999942303, -0.006099381484091282, 0.011049374006688595, -0.009474520571529865, -0.003559295553714037, 0.0025654868222773075, -0.011112876236438751, 0.0020908082369714975, 0.0012097159633412957, -0.03284331038594246, 0.039422135800123215, -0.02761073410511017, 0.011138277128338814, -0.007683760020881891, -0.016015242785215378, -0.01761549711227417, 0.012744881212711334, 0.020231785252690315, 0.014554692432284355, 0.006321638822555542, -0.00028079855837859213, -0.004432450048625469, -0.01579933427274227, 0.011157327331602573, -0.016599461436271667, -0.008991904556751251, 0.00029667411581613123, -0.025743771344423294, 0.010274647735059261, -0.028194954618811607, 0.004133990034461021, 0.013792666606605053, -0.03749166801571846, -0.018047310411930084, -0.015011908486485481, -0.0061946348287165165, -0.02120971865952015, -0.02300

In [6]:
## testing the embedding on a short sentence 
embeddings_2 =get_embedding('This is just a test sentence',engine = "asmitaembedding")
print(embeddings_2)
print(f"Length of the array size :{len(embeddings_2)} ")

[-0.014259698800742626, 0.006637915503233671, 0.00405046995729208, -0.002231272403150797, -0.01043283473700285, 0.02092955820262432, -0.021006222814321518, -0.009468133561313152, -0.014055258594453335, -0.03580257669091225, 0.015933552756905556, 0.0184379443526268, -0.010860881768167019, 0.006021400447934866, 0.003989776596426964, -0.00521002896130085, 0.030538244172930717, -0.00674013514071703, 0.009857847355306149, -0.0028877167496830225, 0.009979233145713806, -0.007640949450433254, -0.022450082004070282, 0.018821269273757935, -0.03069157339632511, -0.007123460527509451, 0.011646698229014874, -0.026066115126013756, 0.009423412382602692, 0.007410954684019089, 0.008158438839018345, -0.009378690272569656, 0.009123140946030617, -0.025836121290922165, -0.001312090316787362, -0.0065484726801514626, -0.010362558998167515, -0.020392902195453644, 0.02370227687060833, -0.019115151837468147, 0.01825905777513981, 0.022309528663754463, -0.007008463144302368, -0.03184155002236366, 0.00024576738360

# now , to understand , how many tokens, we have in our document , we apply tiktoken , 
# wher we can say we can use this type of encoding 

In [7]:
tokenizer = tiktoken.get_encoding("cl100k_base")
sample_encode = tokenizer.encode(df["description"][51]) 
print("No of tokens: ", len(sample_encode))

No of tokens:  29


In [8]:
tokenizer.decode_tokens_bytes(sample_encode)

[b'AP',
 b' -',
 b' From',
 b' LSU',
 b' at',
 b' No',
 b'.',
 b' ',
 b'1',
 b' to',
 b' Ohio',
 b' State',
 b' at',
 b' No',
 b'.',
 b' ',
 b'10',
 b',',
 b' The',
 b' AP',
 b' women',
 b"'s",
 b' basketball',
 b' poll',
 b' had',
 b' no',
 b' changes',
 b' Monday',
 b'.']

# now , we will calculate the  embeddings for each description column, by ffilling up a  new column in existng dataframe

Instead of recalculating all the embeddings for the data, you can make use of the preloaded dataset available in the pickle file.
Do save your embeddings after calculating them, especially if you are converting a big data set, otherwise you will have a cost each time you need it.

In [10]:
df['embedding'] = df["description"].apply(lambda x : get_embedding(x, engine = 'asmitaembedding'))

In [12]:
pd.to_pickle(df, "data/data-embedding_output01102023.pkl")

In [11]:
## when we run the code  , from 2nd iteration onwards  , we need to  just refer the above pickle file 
## so that recalucation of embeddings are not reqyired repetatedly 

In [13]:
df_pickle = pd.read_pickle("data/data-embedding_output01102023.pkl")
df_pickle.head()

,title,description,label_int,label,embedding
0,World Briefings,BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime M...,1,World,"[-0.010669018141925335, -0.024026192724704742,..."
1,Nvidia Puts a Firewall on a Motherboard (PC Wo...,PC World - Upcoming chip set will include buil...,4,Sci/Tech,"[0.0045676459558308125, -0.004495817236602306,..."
2,"Olympic joy in Greek, Chinese press",Newspapers in Greece reflect a mixture of exhi...,2,Sports,"[-0.004999000113457441, 0.007488568313419819, ..."
3,U2 Can iPod with Pictures,"SAN JOSE, Calif. -- Apple Computer (Quote, Cha...",4,Sci/Tech,"[-0.0030229880940169096, -0.021461213007569313..."
4,The Dream Factory,"Any product, any shape, any size -- manufactur...",4,Sci/Tech,"[-0.024735869839787483, -0.009320170618593693,..."


This functions will calculate the cosine similarity between our query and the dataset.

In [17]:
def search_docs(df, user_query:str, engine:str, top_n=3, print_to=True):
    embedding = get_embedding(
        user_query,
        engine=engine
    )
    df["similarities"] = df.embedding.apply(lambda x: cosine_similarity(x, embedding))

    res = (
        df.sort_values("similarities", ascending=False)
        .head(top_n)
    )
    print(f"printing the res :{res}")
    if print_to:
        display(res)
    return res

In [21]:
res = search_docs(df, "who is Tony Blair", top_n=4, engine="asmitaembedding", print_to=True)
for i in res.index:
    print(df["description"][i])
    print("=====================================")

,title,description,label_int,label,embedding,similarities
991,Blair win would endorse war: PM,THE re-election of British Prime Minister Tony...,1,World,"[-0.021780824288725853, -0.008044708520174026,...",0.833302
1396,Blair faces vote amidst hostage crisis,The anguish of hostage Kenneth Bigley in Iraq ...,1,World,"[-0.008130433969199657, -0.01296935323625803, ...",0.818744
0,World Briefings,BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime M...,1,World,"[-0.010669018141925335, -0.024026192724704742,...",0.817152
1618,Blair defends new exam proposals,Tony Blair talks to business leaders about new...,1,World,"[-0.0176305640488863, 0.0022692156489938498, -...",0.807797


THE re-election of British Prime Minister Tony Blair would be seen as an endorsement of the military action in Iraq, Prime Minister John Howard said today.
The anguish of hostage Kenneth Bigley in Iraq hangs over Prime Minister Tony Blair today as he faces the twin test of a local election and a debate by his Labour Party about the divisive war.
BRITAIN: BLAIR WARNS OF CLIMATE THREAT Prime Minister Tony Blair urged the international community to consider global warming a dire threat and agree on a plan of action to curb the quot;alarming quot; growth of greenhouse gases.
Tony Blair talks to business leaders about new proposals for a major shake-up of the English exam system.


In [23]:
## checking anothe result 
#The Instinet Group

res = search_docs(df, "What is Instinet Group ?", top_n=4, engine="asmitaembedding", print_to=True)
for i in res.index:
    print(df["description"][i])
    print("=====================================")

,title,description,label_int,label,embedding,similarities
44,Owner of Big Electronic Stock Trading System I...,"The Instinet Group, owner of one of the larges...",3,Business,"[-0.02767840586602688, -0.01098182238638401, -...",0.868206
60,"Group Formed to Track, Thwart IM Threats",A group of Internet security and instant messa...,4,Sci/Tech,"[-0.02760486491024494, -0.018385833129286766, ...",0.789836
961,"Despite Google's Glitter, Other Dot-Coms Face ...",Reuters - Internet stocks are\as volatile as e...,4,Sci/Tech,"[-0.012919257394969463, -0.02452130988240242, ...",0.780236
1706,"Intel Sees Web Buckling, Invests in New Systems",SAN FRANCISCO (Reuters) - Intel Corp. &lt;A HR...,4,Sci/Tech,"[0.0017545203445479274, -0.016835203394293785,...",0.777815


The Instinet Group, owner of one of the largest electronic stock trading systems, is for sale, executives briefed on the plan say.
A group of Internet security and instant messaging providers have teamed up to detect and thwart the growing threat of IM and peer-to-peer viruses and worms, they said this week.
Reuters - Internet stocks are\as volatile as ever, with growth-starved investors flocking to\the sector in the hope they've bought shares in the next online\blue chip.
SAN FRANCISCO (Reuters) - Intel Corp. &lt;A HREF="http://www.reuters.co.uk/financeQuoteLookup.jhtml?ticker=INTC.O qtype=sym infotype=info qcat=news"&gt;INTC.O&lt;/A&gt; on Thursday outlined its vision of the Internet of the future, one in which millions of computer servers would analyze and direct network traffic to make the Web safer and more efficient.


In [24]:
res = search_docs(df, "Which Israeli helicopter fired missles  in Gaza city and what was the time ?", top_n=4, engine="asmitaembedding", print_to=True)
for i in res.index:
    print(df["description"][i])
    print("=====================================")

,title,description,label_int,label,embedding,similarities
960,Israeli Helicopter Fires at Gaza Building (AP),AP - An Israeli helicopter fired two missiles ...,1,World,"[-0.017681945115327835, -0.003165326314046979,...",0.889791
8,Israeli Helicopter Fires Missile in Gaza -Witn...,GAZA (Reuters) - An Israeli helicopter fired a...,1,World,"[-0.028363429009914398, 0.00033025522134266794...",0.881045
509,"Car Explodes in Gaza City, One Dead",An explosion tore apart a car in Gaza City Mon...,1,World,"[-0.018749656155705452, 0.0009603912476450205,...",0.851953
1538,Israeli Missile Strike Kills Two Gaza Militant...,Reuters - An Israeli missile strike killed at ...,1,World,"[-0.0216471366584301, -0.009843820706009865, 0...",0.851676


AP - An Israeli helicopter fired two missiles in Gaza City after nightfall Wednesday, one at a building in the Zeitoun neighborhood, witnesses said, setting a fire.
GAZA (Reuters) - An Israeli helicopter fired a missile into a town in the southern Gaza Strip late on Wednesday, witnesses said, hours after a Palestinian suicide bomber blew herself up in Jerusalem, killing two Israeli border policemen.
An explosion tore apart a car in Gaza City Monday, killing at least one person, Palestinian witnesses said. They said Israeli warplanes were circling overhead at the time of the blast, indicating a possible missile strike.
Reuters - An Israeli missile strike killed at least\two Hamas gunmen in Gaza City Monday, a day after a top\commander of the Islamic militant group was killed in a similar\attack, Palestinian witnesses and security sources said.
